#0. Preliminary

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_trf

#1. Code

In [ ]:
import pandas as pd
import json
import spacy
from tqdm import notebook, tqdm

In [ ]:
nlp=spacy.load("en_core_web_trf")

In [ ]:
tmp = []
with open('/content/gdrive/MyDrive/PIQA/train.jsonl') as f:
    for obj in f:
        tmp.append(json.loads(obj))

piqa_pair = pd.DataFrame(tmp)
display(piqa_pair.head())

In [ ]:
answer = []
with open('/content/gdrive/MyDrive/PIQA/train-labels.lst') as f:
    answer += f.read().splitlines()

piqa_pair['answer'] = answer # If answer is 0, sol1 is correct, else sol2 is correct
piqa_pair['sol'] = piqa_pair.apply(lambda x: x['sol1'] if x['answer'] == '0' else x['sol2'], axis=1)
piqa_pair.drop(['sol1', 'sol2', 'answer'], axis=1, inplace=True)

display(piqa_pair.head())

In [ ]:
piqa_pair['qa'] = piqa_pair.apply(lambda x: x['goal'] + ' / ' + x['sol'], axis=1)

In [ ]:
display(piqa_pair)

In [ ]:
with open('/content/gdrive/MyDrive/outputs/PIQA-Verb/all.txt') as f:
    piqa_verb = f.read().splitlines()

clean tires같이 clean류 문장은 잘 바뀌지 않음.
goal에 동사가 없는 경우에는 직접 확인 및 변경

In [ ]:
piqa_pair['goal_verbs']=None
piqa_pair['sol_verbs']=None

piqa_pair_length=len(piqa_pair)
i=0
for row in notebook.tqdm(piqa_pair.itertuples(), total=piqa_pair_length):
  col=nlp(row.goal)
  col2=nlp(row.sol)
  tmp=set()
  tmp2=set()
  tmp.update([token.lemma_ for token in col if token.pos_ == 'VERB'])
  tmp2.update([word for word in tmp if word in piqa_verb])

  tmp3=set()
  tmp4=set()
  tmp3.update([token.lemma_ for token in col2 if token.pos_ == 'VERB'])
  tmp4.update([word for word in tmp3 if word in piqa_verb])

  piqa_pair.at[row.Index, 'goal_verbs']=tmp2
  piqa_pair.at[row.Index, 'sol_verbs']=tmp4
  i+=1
display(piqa_pair)

In [ ]:
piqa_pair.drop(['goal', 'sol'], axis=1, inplace=True)
piqa_pair.rename(columns={'qa':'sen'})

sauté
purée

In [ ]:
display(piqa_pair)

In [ ]:
piqa_pair.to_csv("/content/gdrive/MyDrive/outputs/PIQA-Pair-new.csv", index=False, header=False)